<a href="https://colab.research.google.com/github/safdarhayatawan/ATM-App/blob/main/pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
# import os
# from google.colab import userdata
# from pinecone import Pinecone, ServerlessSpec

# # Retrieve Pinecone API Key
# pinecone_api_key = userdata.get('PINECONE_API_KEY')

# # Debugging: Print API key to check if it's retrieved correctly (Remove in production)
# print("Pinecone API Key:", pinecone_api_key)  # Debugging: Ensure API key is retrieved

# if not pinecone_api_key:
#     raise ValueError("Pinecone API Key is missing. Please check if it is set properly.")

# # Initialize Pinecone
# pc = Pinecone(api_key=pinecone_api_key)


Pinecone API Key: pcsk_2XXaA9_GzwVNDcudmKVeCicVRMgrpfYgCg1nA71LcuNDAKgysapgZ19VHRWjqEGif2w5YT


In [ ]:
import os
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

# Try fetching from userdata first
pinecone_api_key = userdata.get('PINECONE_API_KEY')
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)


In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec

index_name = "langchain-safdar-index"  # Change if needed
dimension = 768
metric = "cosine"
cloud = "aws"
region = "us-east-1"

# Get a list of existing indexes
existing_indexes = [index["name"] for index in pc.list_indexes()]

if index_name in existing_indexes:
    print(f"Index '{index_name}' already exists. Connecting...")
else:
    print(f"Creating index: {index_name}...")

    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud=cloud, region=region),
    )

    # Wait for the index to be created
    while index_name not in [idx["name"] for idx in pc.list_indexes()]:
        print("Waiting for index to be created...")
        time.sleep(2)

    print(f"Index '{index_name}' created successfully.")

# Connect to the existing or newly created index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")



Index 'langchain-safdar-index' already exists. Connecting...
Connected to index: langchain-safdar-index


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata
# userdata.get('GOOGLE_API_KEY')
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [ ]:
vector = embeddings.embed_query("we are building a rag")
print(vector[:5])


[0.020280562341213226, -0.00720174890011549, -0.044231660664081573, -0.022845320403575897, -0.0029758561868220568]


In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

Once you have created your vector store, we can interact with it by adding and deleting different items.

Add items to vector store
We can add items to our vector store by using the add_documents function.

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['5857a5ca-cd28-4122-b6a7-02c40a49cddd',
 'fdd0ff39-9e10-45b4-8a87-74f8dc2ed61b',
 'f4a92d35-1d8a-4f89-884b-f39e7440f3e0',
 '522c9b30-3cf3-4ee0-b5db-ae91dfecf704',
 '9bda3367-4f53-436d-b338-91400baeb4c7',
 '6615a582-3fde-4711-8877-f5cfa231ed3c',
 '589e224e-e410-462c-876f-f6e1f491ce1e',
 'b7b5d994-d31b-41c8-97c6-036ff3ffc615',
 'b8f41778-55c3-41ee-8867-2bee72fd880b',
 'd99e25d2-bb21-4b34-b57b-6c055f5cbd1a']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy"
    # k=2,
    # filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
# // ... existing code ...

from langchain_google_genai import GoogleGenerativeAI

# Initialize Google's Generative AI model
model = GoogleGenerativeAI(
    model="gemini-pro",
    api_key=os.getenv("GOOGLE_API_KEY")
)

def answer_user(query: str):
    # Get relevant documents
    results = vector_store.similarity_search_with_score(query, k=2)

    # Format the context from retrieved documents
    context = "\n".join([f"Document {i+1}: {doc.page_content}"
                        for i, (doc, score) in enumerate(results)])

    # Create the prompt
    prompt = f"""Based on the following context, answer the query.

Context:
{context}

Query: {query}

Answer:"""

    # Get response from the model
    response = model.invoke(prompt)
    return response

# Example usage
query = "What's the weather forecast for tomorrow?"
print(answer_user(query))

Cloudy and overcast, with a high of 62 degrees.
